In [6]:
import os

import tensorflow as tf

from lib.dataset.parse_heightmap import parse_heightmap
from lib.ai.model_rainfallwater_mono import model_rainfallwater_mono
from lib.ai.helpers.make_callbacks import make_callbacks
from lib.ai.helpers.summarywriter import summarywriter

In [7]:
filepath_heightmap="/mnt/research-data/main/terrain50-nimrodsized.json.gz"

dir_output = "/tmp/x/mono_segment_TEST"
if not os.path.exists(os.path.join(dir_output, "checkpoints")):
	os.makedirs(os.path.join(dir_output, "checkpoints"))

In [8]:
model = model_rainfallwater_mono(
	metadata={ "rainfallradar": [ 1, 64, 64 ] },
	model_arch_dec="convnext_i_xxtiny"
)

summarywriter(model, filepath_output=os.path.join(dir_output, "summary.txt"))

RAINFALL channels 1 width 64 height 64 HEIGHTMAP_INPUT False
convnext:shape IN x (None, 64, 64, 1)
DEBUG:convnext shape x (None, 64, 64, 1)
DEBUG:model ENCODER output_shape (None, 512)
DEBUG:model BOTTLENECK:stack2image output_shape (None, 4, 4, 512)


2022-12-09 17:35:07.608 | WARNING  | lib.ai.model_rainfallwater_mono:model_rainfallwater_mono:70 - Warning: TODO implement attention from https://ieeexplore.ieee.org/document/9076883
2022-12-09 17:35:07.656 | INFO     | lib.ai.model_rainfallwater_mono:model_rainfallwater_mono:80 - learning_rate: None


In [9]:
heightmap = parse_heightmap(filepath_heightmap) / 100
heightmap = tf.image.crop_to_bounding_box(tf.expand_dims(heightmap, axis=-1), 0, 0, 64, 64)
heightmap_labels = tf.one_hot(tf.cast(tf.math.greater(tf.squeeze(heightmap)/10, 0.05), dtype=tf.int32), 2)

dataset = tf.data.Dataset.from_tensor_slices([heightmap])
dataset_labels = tf.data.Dataset.from_tensor_slices([heightmap_labels])

for item in dataset_labels:
	print("cells", 64*64, "cells/2", (64*64)/2, "shape+", item.shape, tf.math.reduce_sum(tf.math.argmax(item, axis=-1)))
	break
dataset = tf.data.Dataset.zip((dataset_labels, dataset_labels)) \
	.repeat(64 * 64) \
	.batch(64) \
	.prefetch(tf.data.AUTOTUNE)


cells 4096 cells/2 2048.0 shape+ (64, 64, 2) tf.Tensor(2401, shape=(), dtype=int64)


In [10]:
model.fit(
	dataset,
	epochs=50,
	callbacks=make_callbacks("/tmp/x/mono_segment_TEST", model)
)

Epoch 1/50


ValueError: in user code:

    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/bryan-smithl/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 248, in assert_

In [ ]:
prediction = model.predict(tf.expand_dims(heightmap, axis=0))
print(tf.math.argmax(prediction, axis=-1))
print(tf.math.reduce_sum(tf.math.argmax(prediction, axis=-1)))
print(prediction)
